### Import Data and Libraries 

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# Path to folder
path = r'C:\Users\Kripamahesh\Documents\08-06-2023 Instcart Basket Analysis'

### Step 3: Import the customers.csv Dataset

In [3]:
# Import Data set
df_customers =pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)


### Step 4 : Wrangle the Data

In [4]:
# Check the import data with col names
df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
# Checking the size
df_customers.shape

(206209, 10)

In [6]:
# Checking the basic info
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


#### Observations: First Name has only 194950 entries

In [7]:
# Reconfirming using value.counts fun
df_customers['First Name'].value_counts(dropna = False)

NaN        11259
Marilyn     2213
Barbara     2154
Todd        2113
Jeremy      2104
           ...  
Merry        197
Eugene       197
Garry        191
Ned          186
David        186
Name: First Name, Length: 208, dtype: int64

#### Observations: First Names has 11259 missing values 

In [8]:
# checking the descriptive values
df_customers.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [9]:
df_customers.columns

Index(['user_id', 'First Name', 'Surnam', 'Gender', 'STATE', 'Age',
       'date_joined', 'n_dependants', 'fam_status', 'income'],
      dtype='object')

#### Observations: Can rename surnam to 'Last Name' and STATE to 'State'

In [10]:
# Rename 
df_customers.rename(columns={'Surnam' : 'Last Name','STATE' : 'State'}, inplace=True)

### Step 5 : Fundamental Data Quality and Consistency checks

In [11]:
# Check for mixed data types 

for col in df_customers.columns.tolist():
        weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
        if len (df_customers[weird]) > 0:
            print(col)

First Name


In [12]:
#### Observations :  First Name had mixed data types, hence change to 'Str'

df_customers['First Name'] = df_customers['First Name'].astype('str')

In [13]:
# Recheck for mixed data types

for col in df_customers.columns.tolist():
        weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
        if len (df_customers[weird]) > 0:
            print(col)

In [14]:
# Chck for missing values 

df_customers.isnull().sum()

user_id         0
First Name      0
Last Name       0
Gender          0
State           0
Age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

In [15]:
# Check for duplicates

df_dups = df_customers[df_customers.duplicated()]

In [16]:
df_dups

,user_id,First Name,Last Name,Gender,State,Age,date_joined,n_dependants,fam_status,income


#### Observations: There are no missing vales and duplicates

### Step 6: Combine the 2 Data frames 

In [18]:
# Import the orders_products_aggregated.pkl dataset 

ords_prods_merge =pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_aggregated.pkl'))

In [19]:
# Check the info
ords_prods_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434212 entries, 0 to 32434211
Data columns (total 29 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   order_id               int64   
 1   user_id                int64   
 2   eval_set               object  
 3   order_number           int64   
 4   order_dow              int64   
 5   order_hour_of_day      int64   
 6   days_after_last_order  float64 
 7   product_id             int64   
 8   add_to_cart_order      int64   
 9   reordered              int64   
 10  Unnamed: 0             int64   
 11  product_name           object  
 12  aisle_id               int64   
 13  department_id          int64   
 14  prices                 float64 
 15  _merge                 category
 16  prices_range_loc       object  
 17  price_range_loc        object  
 18  busiest_day            object  
 19  busiest_days           object  
 20  busiest_period_of_day  object  
 21  max_order              int64 

In [20]:
# Checking the size
ords_prods_merge.shape

(32434212, 29)

In [21]:
#delete the _merge flag
del ords_prods_merge['_merge']

In [ ]:
# Combine the datasets based on user_id column

### df_combined= pd.merge(df_customers, ords_prods_merge, on= 'user_id')

In [22]:
ords_prods_merge.drop(columns = {'eval_set', 'Unnamed: 0'}, inplace = True)

In [23]:
# Change dtypes from ords_prods_merge dataframe

ords_prods_merge['order_id']=ords_prods_merge['order_id'].astype('int32')

In [24]:
ords_prods_merge['user_id']=ords_prods_merge['user_id'].astype('int32')

In [25]:
ords_prods_merge['order_number']=ords_prods_merge['order_number'].astype('int32')

In [26]:
ords_prods_merge['order_dow']=ords_prods_merge['order_dow'].astype('int32')

In [27]:
ords_prods_merge['order_hour_of_day']=ords_prods_merge['order_hour_of_day'].astype('int32')

In [28]:
ords_prods_merge['days_after_last_order']=ords_prods_merge['days_after_last_order'].astype('float16')

In [29]:
ords_prods_merge['product_id']=ords_prods_merge['product_id'].astype('int32')

In [30]:
ords_prods_merge['reordered']=ords_prods_merge['reordered'].astype('int8')

In [31]:
ords_prods_merge['add_to_cart_order']=ords_prods_merge['add_to_cart_order'].astype('int32')

In [32]:
ords_prods_merge['user_id']=ords_prods_merge['user_id'].astype('int32')

In [33]:
ords_prods_merge['aisle_id']=ords_prods_merge['aisle_id'].astype('int8')

In [34]:
ords_prods_merge['department_id']=ords_prods_merge['department_id'].astype('int8')

In [35]:
ords_prods_merge['prices']=ords_prods_merge['prices'].astype('float16')

In [36]:
ords_prods_merge['user_id']=ords_prods_merge['user_id'].astype('int32')

In [37]:
# change dtypes from df_customers dataframe
df_customers['Age']=df_customers['Age'].astype('int8')

In [38]:
df_customers['n_dependants']=df_customers['n_dependants'].astype('int8')

In [39]:
df_customers['income']=df_customers['income'].astype('int8')

In [44]:
ords_prods_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434212 entries, 0 to 32434211
Data columns (total 26 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   order_id               int32  
 1   user_id                int32  
 2   order_number           int32  
 3   order_dow              int32  
 4   order_hour_of_day      int32  
 5   days_after_last_order  float16
 6   product_id             int32  
 7   add_to_cart_order      int32  
 8   reordered              int8   
 9   product_name           object 
 10  aisle_id               int8   
 11  department_id          int8   
 12  prices                 float16
 13  prices_range_loc       object 
 14  price_range_loc        object 
 15  busiest_day            object 
 16  busiest_days           object 
 17  busiest_period_of_day  object 
 18  max_order              int64  
 19  loyalty_flag           object 
 20  avg_price              float64
 21  spending_flag          object 
 22  mean_price      

In [40]:
# Combine the datasets based on user_id column
df_combined= pd.merge(df_customers, ords_prods_merge, on= 'user_id')

In [41]:
df_combined.shape

(32434212, 35)

### Step 8 Export the dataframe as file


In [42]:
# Export 
df_combined.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_customer_merged.pkl'))